In [110]:
#######################################################################################################################
## Script: AGOL_2_SQL.py
## Author: Jeremy Mullins, Derek Morgan
## Date:
##
## Description:
##        This script is the test script for importing a
##        AGOL feature layer into MS SQL
##
## Required prerequisites:
##        - ArcGIS API for Python
##            (https://developers.arcgis.com/python/)
##
##        - pyodbc Module
##            (https://github.com/mkleehammer/pyodbc)
##                to install after download, open Python command prompt and type the following:
##                    -- pip install pyodbc --
##
##        - MS ODBC Driver for SQL Server (v17)
##            (https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-2017)
########################################################################################################################

In [127]:
# import all necessary modules
from arcgis.gis import GIS, Item
import os
import pyodbc
import zipfile

In [112]:
# connect to ArcGIS Online Account
gis = GIS(profile='jwm35')

In [128]:
# get feature layer in question
uwf_buildings = gis.content.get('ea0c379c597c409e843939710add43f2')
uwf_layers = uwf_buildings.layers
uwf_build = uwf_layers[0]

In [130]:
# export feature layer into CSV and save locally as ZIP file
output_file = uwf_buildings.export(title='uwfbuildings', export_format="CSV")
output_file.download(r'W:\src')

'W:\\src\\uwfbuildings.zip'

In [132]:
# extract ZIP to local folder and delete ZIP file
with zipfile.ZipFile(r'W:\src\uwfbuildings.zip','r') as zip_ref:
    zip_ref.extractall(r'W:\src')
zip_ref.close

#### need to delete CSV collection from AGOL
os.remove(r'W:\src\uwfbuildings.zip')

In [133]:
# import CSV into SQL database as view
con = pyodbc.connect(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=MINASTIRITH;DATABASE=src;Trusted_Connection=yes;')
cur = con.cursor()